In [32]:
import sys
sys.path.append("/u/lukas.rilling/dev/")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import h5py
%matplotlib widget

from returnn_training_progress import get_epoch_data
from returnn_training_plot_nb import plot_df
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
globs = [
    "/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/tts_architecture/glow_tts/raw_audio/decoder_test/enc768/mean_only/*/forward/output/output.hdf",
    "/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/tts_architecture/glow_tts/raw_audio/decoder_test/enc768/with_sigma/*/forward/output/output.hdf",
    "/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/librispeech_glow_asr/pytorch/encoding_test/enc768/mean_only/glowTTS_encoding_test_*/forward/output/output.hdf",
    "/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/librispeech_glow_asr/pytorch/encoding_test/enc768/with_sigma/glowTTS_encoding_test_*/forward/output/output.hdf",
    # "/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/tts_architecture/glow_tts/raw_audio/decoder_test/enc192/*/forward/output/output.hdf",
    # "/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/librispeech_glow_asr/pytorch/encoding_test/enc192/*_ce/forward/output/output.hdf",
    # "/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/librispeech_glow_asr/pytorch/encoding_test/enc192/*_mean_only/forward/output/output.hdf",
    # "/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/librispeech_glow_asr/pytorch/encoding_test/enc192/*_blstm/forward/output/output.hdf",
]
lr_files = []
for g in globs:
    lr_files += glob.glob(g)

print(lr_files)
exlude = "speaker_drop"
lr_files = [l for l in lr_files if not exlude in l]
# print(lr_files)
common_prefix = os.path.commonpath(lr_files)
common_sufix = os.path.commonpath([l[::-1] for l in lr_files])[::-1]

names = []
for f in lr_files:
    names.append(f.removeprefix(common_prefix).removesuffix(common_sufix))

files = dict(zip(lr_files, names))
files, len(lr_files)

['/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/tts_architecture/glow_tts/raw_audio/decoder_test/enc768/mean_only/glowTTS_encoder_sample_test_simple_linear/forward/output/output.hdf', '/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/tts_architecture/glow_tts/raw_audio/decoder_test/enc768/mean_only/glowTTS_encoder_sample_test_maxlike_alignment_blstm/forward/output/output.hdf', '/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/tts_architecture/glow_tts/raw_audio/decoder_test/enc768/mean_only/glowTTS_decoder_test_multi_layer_ffn/forward/output/output.hdf', '/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/tts_architecture/glow_tts/raw_audio/decoder_test/enc768/mean_only/glowTTS_encoder_sample_test_multi_layer_ffn/forward/output/output.hdf', '/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/tts_architecture/glow_tts/raw_audio/decoder_test/enc768/mean_only/gl

({'/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/tts_architecture/glow_tts/raw_audio/decoder_test/enc768/mean_only/glowTTS_encoder_sample_test_simple_linear/forward/output/output.hdf': '/tts_architecture/glow_tts/raw_audio/decoder_test/enc768/mean_only/glowTTS_encoder_sample_test_simple_linear/',
  '/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/tts_architecture/glow_tts/raw_audio/decoder_test/enc768/mean_only/glowTTS_encoder_sample_test_maxlike_alignment_blstm/forward/output/output.hdf': '/tts_architecture/glow_tts/raw_audio/decoder_test/enc768/mean_only/glowTTS_encoder_sample_test_maxlike_alignment_blstm/',
  '/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/tts_architecture/glow_tts/raw_audio/decoder_test/enc768/mean_only/glowTTS_decoder_test_multi_layer_ffn/forward/output/output.hdf': '/tts_architecture/glow_tts/raw_audio/decoder_test/enc768/mean_only/glowTTS_decoder_test_multi_layer_ffn/',
  '/u/luk

In [35]:
replace_dict = {
    "librispeech_glow_asr/pytorch/encoding_test/": "",
    "tts_architecture/glow_tts/raw_audio/decoder_test/": "",
    "/": " | ",
    "_encoder_sample_test": " Sampled ",
    "_decoder_test": " Decoder output ",
    # "_encoding_test_blstm": "1x512 BLSTM  Mean + log(std)",
    "_encoding_test": "",
    "_ce": "Mean + log(std)",
    "_mean_only": "Mean",
    "_encoding_test_mean_only": "Mean",
    "glowTTS": "",
    "_maxlike_alignment": " MAS ",
    "_simple_linear": " 1x80 linear ",
    "_multi_layer_ffn": " 3x512 linear ",
    "_blstm": " 1x512 BLSTM ",
    "tts_architecture": "",
    "enc192": "192 channels",
    "enc768": "768 channels",
}

In [36]:
import re

def move_pattern_to_beginning(input_string, pattern):
    match = re.search(pattern, input_string)
    if match:
        matched_text = match.group(0)
        updated_string = matched_text + ' ' + input_string.replace(matched_text, '', 1).strip()
        return updated_string
    else:
        return input_string

pattern = r' \d+x\s*([a-zA-Z]| )*\d+ ([a-zA-Z])* '

In [37]:
new_files = {}
# print(files)
for file, name in files.items():
    n = name
    for k, v in replace_dict.items():
        n = n.replace(k, v)
        print(n)
    n = move_pattern_to_beginning(n, pattern)
    new_files[file] = n

new_files

/tts_architecture/glow_tts/raw_audio/decoder_test/enc768/mean_only/glowTTS_encoder_sample_test_simple_linear/
/enc768/mean_only/glowTTS_encoder_sample_test_simple_linear/
 | enc768 | mean_only | glowTTS_encoder_sample_test_simple_linear | 
 | enc768 | mean_only | glowTTS Sampled _simple_linear | 
 | enc768 | mean_only | glowTTS Sampled _simple_linear | 
 | enc768 | mean_only | glowTTS Sampled _simple_linear | 
 | enc768 | mean_only | glowTTS Sampled _simple_linear | 
 | enc768 | mean_only | glowTTS Sampled _simple_linear | 
 | enc768 | mean_only | glowTTS Sampled _simple_linear | 
 | enc768 | mean_only |  Sampled _simple_linear | 
 | enc768 | mean_only |  Sampled _simple_linear | 
 | enc768 | mean_only |  Sampled  1x80 linear  | 
 | enc768 | mean_only |  Sampled  1x80 linear  | 
 | enc768 | mean_only |  Sampled  1x80 linear  | 
 | enc768 | mean_only |  Sampled  1x80 linear  | 
 | enc768 | mean_only |  Sampled  1x80 linear  | 
 | 768 channels | mean_only |  Sampled  1x80 linear  | 
/tts

{'/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/tts_architecture/glow_tts/raw_audio/decoder_test/enc768/mean_only/glowTTS_encoder_sample_test_simple_linear/forward/output/output.hdf': ' 1x80 linear  | 768 channels | mean_only |  Sampled  |',
 '/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/tts_architecture/glow_tts/raw_audio/decoder_test/enc768/mean_only/glowTTS_encoder_sample_test_maxlike_alignment_blstm/forward/output/output.hdf': ' 1x512 BLSTM  | 768 channels | mean_only |  Sampled  MAS  |',
 '/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/tts_architecture/glow_tts/raw_audio/decoder_test/enc768/mean_only/glowTTS_decoder_test_multi_layer_ffn/forward/output/output.hdf': ' 3x512 linear  | 768 channels | mean_only |  Decoder output  |',
 '/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/tts_architecture/glow_tts/raw_audio/decoder_test/enc768/mean_only/glowTTS_encoder_sample_tes

In [38]:
sorting_definition_glow_model = ["192 channels", "768 channels"]
sorting_definition_model = [" 1x80 linear ", " 3x512 linear ", " 1x512 BLSTM "]
sorting_definition_mean = ["mean_only", "with_sigma"]
sorting_definition_input = ["Mean |", "Mean + log(std) |", " Sampled  |", " Sampled  MAS  |", " Decoder output  |"]
def sorting_function(a):
    if "mean_only" in a[1] or "with_sigma" in a[1]:
        split_a = a[1].split(" | ")

        sorting_index_a = sorting_definition_model.index(split_a[0]) * 1e3

        if "channels" in a[1]:
            sorting_index_a += sorting_definition_glow_model.index(split_a[1]) * 1e2 + sorting_definition_mean.index(split_a[2]) * 1e1 + sorting_definition_input.index(split_a[3])   
        else:
            sorting_index_a += sorting_definition_mean.index(split_a[1]) * 1e1 + sorting_definition_input.index(split_a[2])

    else:
        split_a = a[1].split(" | ")

        sorting_index_a = sorting_definition_model.index(split_a[0]) * 1e2
        if "channels" in a[1]:
            sorting_index_a += sorting_definition_glow_model.index(split_a[1]) * 1e1 + sorting_definition_input.index(split_a[2])
        else:
            sorting_index_a += sorting_definition_input.index(split_a[1])

    return sorting_index_a

for k,v in new_files.items():
    if "with_sigma" in v:
        sorting_function([k,v])
        break

In [39]:
sorted_files = dict(sorted(new_files.items(), key=sorting_function))
sorted_files.values()

dict_values([' 1x80 linear  | 768 channels | mean_only | Mean |', ' 1x80 linear  | 768 channels | mean_only |  Sampled  |', ' 1x80 linear  | 768 channels | mean_only |  Sampled  MAS  |', ' 1x80 linear  | 768 channels | mean_only |  Decoder output  |', ' 1x80 linear  | 768 channels | with_sigma | Mean |', ' 1x80 linear  | 768 channels | with_sigma | Mean + log(std) |', ' 1x80 linear  | 768 channels | with_sigma |  Sampled  |', ' 1x80 linear  | 768 channels | with_sigma |  Sampled  MAS  |', ' 1x80 linear  | 768 channels | with_sigma |  Decoder output  |', ' 3x512 linear  | 768 channels | mean_only | Mean |', ' 3x512 linear  | 768 channels | mean_only |  Sampled  |', ' 3x512 linear  | 768 channels | mean_only |  Sampled  MAS  |', ' 3x512 linear  | 768 channels | mean_only |  Decoder output  |', ' 3x512 linear  | 768 channels | with_sigma | Mean |', ' 3x512 linear  | 768 channels | with_sigma | Mean + log(std) |', ' 3x512 linear  | 768 channels | with_sigma |  Sampled  |', ' 3x512 linear  

In [40]:
for file, name in sorted_files.items():
    data = h5py.File(file)
    epoch_data = get_epoch_data(file.replace("forward/output/output.hdf", "training/work/learning_rates"), 100)
    if "dev_loss_ce" in epoch_data["error"]: 
        dev_loss_ce = epoch_data["error"]["dev_loss_ce"]
    else:
        dev_loss_ce = np.inf
    mean = np.array(data["inputs"]).mean()
    print(f"| {name} {1e2*mean:.2f} | {dev_loss_ce:.3f} |")

|  1x80 linear  | 768 channels | mean_only | Mean | 33.88 | 2.295 |
|  1x80 linear  | 768 channels | mean_only |  Sampled  | 23.12 | 2.967 |
|  1x80 linear  | 768 channels | mean_only |  Sampled  MAS  | 24.49 | 2.936 |
|  1x80 linear  | 768 channels | mean_only |  Decoder output  | 24.45 | 2.943 |
|  1x80 linear  | 768 channels | with_sigma | Mean | 60.46 | 1.284 |
|  1x80 linear  | 768 channels | with_sigma | Mean + log(std) | 64.04 | 1.160 |
|  1x80 linear  | 768 channels | with_sigma |  Sampled  | 33.86 | 2.388 |
|  1x80 linear  | 768 channels | with_sigma |  Sampled  MAS  | 37.53 | 2.289 |
|  1x80 linear  | 768 channels | with_sigma |  Decoder output  | 37.11 | 2.314 |
|  3x512 linear  | 768 channels | mean_only | Mean | 76.11 | 0.775 |
|  3x512 linear  | 768 channels | mean_only |  Sampled  | 23.66 | 2.884 |
|  3x512 linear  | 768 channels | mean_only |  Sampled  MAS  | 24.97 | 2.857 |
|  3x512 linear  | 768 channels | mean_only |  Decoder output  | 24.87 | 2.868 |
|  3x512 linear